In [83]:
import numpy as np
import pandas as pd
import datetime as dt

In [150]:
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [151]:
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(pd.to_datetime(sales['ordered_at']).dt.strftime('%Y-%m-%d %H:%M:%S'))

In [152]:
prices.info()
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   product_id  14 non-null     int64         
 1   old_price   14 non-null     int64         
 2   new_price   14 non-null     int64         
 3   updated_at  14 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 576.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   product_id        175 non-null    int64         
 1   ordered_at        175 non-null    datetime64[ns]
 2   quantity_ordered  175 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.2 KB


In [107]:
print(
    prices.head(10)
)
print(
    sales.head(10)
)

   product_id  old_price  new_price          updated_at
0          64     270000     239000 2018-09-10 16:37:00
1     3954203      60000      64000 2018-09-11 11:54:00
2     3954203      60500      57500 2018-09-17 22:59:00
3     3954203      64000      60500 2018-09-15 03:49:00
4     3954203      68800      60000 2018-09-10 16:32:00
5     3998909      15500      16500 2018-09-16 05:09:00
6     3998909      17000      15500 2018-09-13 06:43:00
7     3998909      19000      17000 2018-09-10 16:35:00
8     4085861      53500      52000 2018-09-17 22:59:00
9     4085861      53500      67000 2018-09-12 03:51:00
   product_id          ordered_at  quantity_ordered
0     3998909 2018-09-18 17:51:00                 1
1     3998909 2018-09-18 12:52:00                 1
2     3998909 2018-09-18 11:33:00                 1
3     3998909 2018-09-18 18:47:00                 1
4     3998909 2018-09-18 17:36:00                 1
5     3998909 2018-09-17 19:09:00                 1
6     3998909 2018-0

In [88]:
prices2 = prices

In [89]:
# prices2 = prices2.sort_values(['product_id', 'updated_at'])

In [153]:
prices = prices.sort_values(['product_id', 'updated_at'])
prices['old_date'] = prices.groupby('product_id')\
                ['updated_at'].shift(1, fill_value= pd.to_datetime('2010-01-01 00:00:00'))
prices = prices.rename(columns= {'updated_at': 'new_date'}).iloc[:,[0,1,2,4,3]]


In [154]:
# prices = prices.sort_values(['product_id', 'new_date'])
print(prices.iloc[1,:])
prices.head(20)

product_id                3954203
old_price                   68800
new_price                   60000
old_date      2010-01-01 00:00:00
new_date      2018-09-10 16:32:00
Name: 4, dtype: object


,product_id,old_price,new_price,old_date,new_date
0,64,270000,239000,2010-01-01 00:00:00,2018-09-10 16:37:00
4,3954203,68800,60000,2010-01-01 00:00:00,2018-09-10 16:32:00
1,3954203,60000,64000,2018-09-10 16:32:00,2018-09-11 11:54:00
3,3954203,64000,60500,2018-09-11 11:54:00,2018-09-15 03:49:00
2,3954203,60500,57500,2018-09-15 03:49:00,2018-09-17 22:59:00
7,3998909,19000,17000,2010-01-01 00:00:00,2018-09-10 16:35:00
6,3998909,17000,15500,2018-09-10 16:35:00,2018-09-13 06:43:00
5,3998909,15500,16500,2018-09-13 06:43:00,2018-09-16 05:09:00
11,4085861,60000,53500,2010-01-01 00:00:00,2018-09-11 08:51:00
9,4085861,53500,67000,2018-09-11 08:51:00,2018-09-12 03:51:00


In [155]:
def return_price(id, time):
    for i in prices.index:
        latest_time = pd.to_datetime('2010-01-01 00:00:00')
        if  (id == prices.product_id[i]) & ((time >= prices.old_date[i]) & (time < prices.new_date[i])):
            price_return = prices.old_price[i]
            break     
        elif (id == prices.product_id[i]) & (time >= prices.new_date[i]):  
            price_return = prices.new_price[i]
    return price_return

In [156]:
id = 3998909
time = pd.to_datetime('2018-09-18 16:35:00')

In [157]:
return_price(id, time)

16500

In [158]:
sales['price'] = pd.Series(dtype='int')

In [159]:
for i in sales.index:
         sales.price[i] = return_price(sales.product_id[i], sales.ordered_at[i])

<ipython-input-159-587b311a6a7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales.price[i] = return_price(sales.product_id[i], sales.ordered_at[i])


In [160]:
sales['revenue'] = sales.quantity_ordered * sales.price

In [98]:
sales.head(20)

,product_id,ordered_at,quantity_ordered,price,revenue
0,3998909,2018-09-18 17:51:00,1,17000.0,17000.0
1,3998909,2018-09-18 12:52:00,1,17000.0,17000.0
2,3998909,2018-09-18 11:33:00,1,17000.0,17000.0
3,3998909,2018-09-18 18:47:00,1,17000.0,17000.0
4,3998909,2018-09-18 17:36:00,1,17000.0,17000.0
5,3998909,2018-09-17 19:09:00,1,17000.0,17000.0
6,3998909,2018-09-18 16:02:00,1,17000.0,17000.0
7,3998909,2018-09-17 22:29:00,1,17000.0,17000.0
8,3998909,2018-09-18 19:46:00,2,17000.0,34000.0
9,3998909,2018-09-18 11:02:00,1,17000.0,17000.0


In [99]:
sales.groupby(['product_id', 'price'])['revenue'].sum()

product_id  price   
64          239000.0     956000.0
3954203     60000.0      240000.0
            64000.0      640000.0
3998909     15500.0       15500.0
            17000.0      272000.0
4085861     53500.0     2140000.0
            58000.0     2204000.0
            60000.0      180000.0
            62500.0     3062500.0
            67000.0      871000.0
Name: revenue, dtype: float64

In [161]:
sales.pivot_table(
    values= 'revenue',
    index= ['product_id', 'price'],
    aggfunc= sum
)

revenue
product_id price              
64         239000.0   956000.0
3954203    57500.0     57500.0
           60000.0    180000.0
           64000.0    640000.0
3998909    15500.0     15500.0
           16500.0    231000.0
           17000.0     34000.0
4085861    52000.0   1040000.0
           53500.0   2140000.0
           58000.0   2204000.0
           60000.0    180000.0
           62500.0   1812500.0
           67000.0    871000.0